## Querying and Organizing Data

In [17]:
import os
import logging

import pandas as pd

from SteamXmlProcessor import SteamXmlProcessor
from MathUtils import MathUtils
from SteamSpyQuery import SteamSpyQuery
from SteamDataBokehGraphGenerator import SteamDataBokehGraphGenerator

In [18]:
# Set up logging (Jupyter sets up it's own so we have to add ours instead of using a basicConfig)
log = logging.getLogger()
fhandler = logging.FileHandler(filename='mylog.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
log.addHandler(fhandler)
log.setLevel(logging.CRITICAL)

In [19]:
def query_steam_data_for_user():
    # Note, your 'Game details' must be set to 'Public' for this to work.
    # This is done in your profile -> Edit Profile -> Privacy Settings -> Game details
    # To find your username, check your profile under General -> Custom URL
    username = ''
    if username == '':
        if os.path.exists("steam_id.dat"):
            log.info('Reading steam ID from file')
            with open("steam_id.dat", "r", encoding="utf-8") as id_file:
                username = id_file.read()
        else:
            raise Exception("Missing steam id")
            log.critical('Need steam user ID')

    cache_file = f"{username}_steam_games.xml"
    if os.path.exists(cache_file):
        xmlProcessor = SteamXmlProcessor.from_file(cache_file)
    else:
        xmlProcessor = SteamXmlProcessor.from_username(
            username, cache_file)

    return xmlProcessor.get_game_infos()   

In [20]:
# Main

# Decorate our steam library info with ranking info from SteamSpy
game_infos = query_steam_data_for_user()
decorated_game_infos = pd.DataFrame.copy(game_infos)
steam_spy_query = SteamSpyQuery()
decorated_game_infos = steam_spy_query.get_data_for_games(decorated_game_infos)
decorated_game_infos = MathUtils.add_bayesian_average_to_gamespy_dataframe(decorated_game_infos)

In [21]:
# Do any filtering or re-arranging you want to here
decorated_game_infos = decorated_game_infos.sort_values(by=['BayesianAverage'], ascending=False)

# DLCs have no ratings, drop them from the list
decorated_game_infos.drop(decorated_game_infos[decorated_game_infos["RatingsRatio"] == 0].index, inplace = True)

# Write to file for easy access
decorated_game_infos.to_csv("decorated_game_infos.csv")

In [22]:
graph_generator = SteamDataBokehGraphGenerator(decorated_game_infos)
graph_generator.generate_most_played_games_graph()
graph_generator.generate_most_played_games_2weeks_graph()
graph_generator.generate_most_played_games_versus_rating_graph()
graph_generator.generate_best_unplayed_games_average()
graph_generator.generate_best_unplayed_games_bayesian_average()

TypeError: SteamDataBokehGraphGenerator.__init__() missing 1 required positional argument: 'decorated_game_infos'